In [1]:
import sys

In [2]:
txt_data = [line.rstrip('\n').rstrip(')').lstrip('(') for line in open('GraphData.txt')]
graph_data = []

# Put graph data into a array of tuples.
for line in txt_data:
    split = line.split(',')
    if len(split) == 2:
        x = int(split[0])
        y = int(split[1])
        graph_data.append((x, y, abs(x - y)))
        
print(graph_data)

[(277, 325, 48), (6499, 6525, 26), (1586, 1688, 102), (1435, 1500, 65), (2817, 2954, 137), (1467, 1550, 83), (4952, 4986, 34), (4393, 4449, 56), (840, 895, 55), (5088, 5154, 66), (3637, 3744, 107), (3618, 3622, 4), (2287, 2422, 135), (5143, 5184, 41), (6468, 6472, 4), (2167, 2183, 16), (3557, 3706, 149), (1441, 1507, 66), (84, 210, 126), (5625, 5870, 245), (6389, 6421, 32), (2279, 2421, 142), (1513, 1528, 15), (5016, 5078, 62), (1634, 1698, 64), (4215, 4474, 259), (4415, 4455, 40), (6327, 6568, 241), (5136, 5175, 39), (3054, 3131, 77), (3553, 3641, 88), (5123, 5138, 15), (928, 1027, 99), (2869, 3036, 167), (41, 144, 103), (4969, 5016, 47), (250, 322, 72), (6461, 6494, 33), (4964, 5096, 132), (3098, 3104, 6), (4242, 4460, 218), (3652, 3720, 68), (31, 264, 233), (740, 902, 162), (1437, 1658, 221), (3582, 3744, 162), (826, 898, 72), (2382, 2433, 51), (1594, 1680, 86), (5670, 5744, 74), (5682, 5749, 67), (5634, 5853, 219), (1509, 1691, 182), (4990, 5019, 29), (915, 1024, 109), (2811, 3054,

In [3]:
class Forest(): 
    def __init__(self, graph_data):         
        self.graph_data = graph_data

        dicts = self.build_graph_dict(self.graph_data)        
        self.graph_dict = dicts[0]
        self.graph_dict_rev = dicts[1]                
        
        self.V = len(self.graph_dict)
        
        self.graph = self.build_adj_list(self.graph_data, self.graph_dict, self.V)
        self.cc = self.connectedComponents()
        
        self.forest = []
        self.forest_data = []
        self.build_connected_forest()

        print("Forest Info:")
        print("\tNumber of vertices:", str(self.V))
        print("\tNumber of edges:", len(self.graph_data))
        print("\tNumber of connected components in forest:", len(self.cc))
        
    def build_graph_dict(self, graph_data):
        graph_dict = {}
        graph_dict_rev = {}
        i = 0        
        for x in graph_data:
            if x[0] not in graph_dict:
                graph_dict[x[0]] = i
                graph_dict_rev[i] = x[0]
                i += 1
            if x[1] not in graph_dict:
                graph_dict[x[1]] = i
                graph_dict_rev[i] = x[1]
                i += 1
        return (graph_dict, graph_dict_rev)
        
    def build_adj_list(self, graph_data, graph_dict, length):
        graph = [[(0, 0) for i in range(length)] for j in range(length)]

        for d in graph_data:
            x = graph_dict[d[0]]
            y = graph_dict[d[1]]
            graph[x][y] = (y, d[2])
            graph[y][x] = (x, d[2])
        
        return graph
            
    def build_connected_forest(self):
        for c in self.cc:
            graph_data = []
            for x in c:
                for edge in self.find_edges(x):
                    if not self.edge_in_edges(edge, graph_data):
                        graph_data.append(edge)   

            d = self.build_graph_dict(graph_data)
            adj = self.build_adj_list(graph_data, d[0], len(d[0]))
            
            self.forest_data.append(graph_data)
            self.forest.append(adj)
                
    def find_edges(self, v):
        edges = []
        for edge in self.graph_data:
            if edge[0] == self.graph_dict_rev[v] or edge[1] == self.graph_dict_rev[v]:
                edges.append(edge)
        return edges
    
    def edge_in_edges(self, edge, edges):
        for e in edges:
            if edge[0] == e[0] and edge[1] == e[1]:
                return True
        return False
            
    def print_cc_list(self):
        i = 0
        for cc in self.forest_data:
            print("\nComponent {0}:".format(str(i)))
            i += 1
            for c in cc:
                s = "["
                for x in c:
                    s += str(x) + ','
                s = s[:-1]
                s += "]"
                print(s)        
            
    def printMST(self, parent): 
        print("Edge \t\tWeight")
        for i in range(1, self.V): 
            print(self.graph_dict_rev[parent[i]], "-", self.graph_dict_rev[i], "\t", self.graph[i][parent[i]][1])

    def minKey(self, key, mstSet): 
        min = sys.maxsize 

        for v in range(self.V): 
            if key[v] < min and mstSet[v] == False: 
                min = key[v] 
                min_index = v 

        return min_index 
    
    def DFSUtil(self, temp, v, visited): 
        # Mark the current vertex as visited 
        visited[v] = True

        # Store the vertex to list 
        temp.append(v) 
        

        # Repeat for all vertices adjacent 
        # to this vertex v 
        for i in self.graph[v]: 
            if visited[i[0]] == False: 

                # Update the list 
                temp = self.DFSUtil(temp, i[0], visited) 
        return temp 

    # Method to retrieve connected components 
    # in an undirected graph 
    def connectedComponents(self): 
        visited = [] 
        cc = [] 
        for i in range(self.V): 
            visited.append(False) 
        for v in range(self.V): 
            if visited[v] == False: 
                temp = [] 
                cc.append(self.DFSUtil(temp, v, visited)) 
        return cc     

    def primMST(self): 
        key = [sys.maxsize] * self.V 
        parent = [None] * self.V

        key[0] = 0 
        mstSet = [False] * self.V 
        parent[0] = -1

        for x in range(self.V): 
            u = self.minKey(key, mstSet) 
            mstSet[u] = True
            for v in range(self.V): 
                if self.graph[u][v][1] > 0 and mstSet[v] == False and key[v] > self.graph[u][v][1]: 
                        key[v] = self.graph[u][v][1] 
                        parent[v] = u 

        self.printMST(parent) 

In [4]:
forest = Forest(graph_data) 

forest.print_cc_list()


#print(g.graph)

Forest Info:
	Number of vertices: 1426
	Number of edges: 2342
	Number of connected components in forest: 10

Component 0:
[277,325,48]
[73,277,204]
[96,277,181]
[251,277,26]
[274,277,3]
[129,277,148]
[274,325,51]
[274,286,12]
[199,274,75]
[11,274,263]
[45,274,229]
[252,274,22]
[274,287,13]
[8,11,3]
[11,289,278]
[11,152,141]
[11,31,20]
[11,74,63]
[31,264,233]
[31,75,44]
[31,172,141]
[31,151,120]
[31,294,263]
[7,264,257]
[156,264,108]
[264,284,20]
[217,264,47]
[210,264,54]
[264,278,14]
[264,288,24]
[84,210,126]
[183,210,27]
[210,319,109]
[153,210,57]
[200,210,10]
[84,87,3]
[84,276,192]
[8,84,76]
[8,18,10]
[8,60,52]
[60,285,225]
[60,120,60]
[60,218,158]
[60,250,190]
[60,300,240]
[60,172,112]
[60,104,44]
[250,322,72]
[115,250,135]
[15,250,235]
[239,322,83]
[5,239,234]
[5,329,324]
[5,114,109]
[5,296,291]
[5,238,233]
[1,329,328]
[1,219,218]
[1,99,98]
[43,99,56]
[99,119,20]
[34,99,65]
[86,99,13]
[99,185,86]
[70,119,49]
[70,120,50]
[70,301,231]
[62,70,8]
[74,120,46]
[120,308,188]
[120,183,63]


[776,843,67]
[843,955,112]
[776,985,209]
[760,1007,247]
[766,1008,242]
[764,1008,244]
[727,886,159]
[886,1020,134]
[727,817,90]
[796,1012,216]
[919,1026,107]
[925,1026,101]
[799,922,123]
[803,1013,210]
[910,1023,113]
[906,1023,117]
[917,1025,108]
[1025,1036,11]
[934,1028,94]

Component 7:
[3637,3744,107]
[3637,3790,153]
[3579,3637,58]
[3582,3744,162]
[3744,3766,22]
[3582,3760,178]
[3582,3751,169]
[3582,3788,206]
[3507,3582,75]
[3577,3760,183]
[3622,3760,138]
[3500,3760,260]
[3618,3622,4]
[3622,3724,102]
[3535,3622,87]
[3618,3657,39]
[3618,3749,131]
[3519,3618,99]
[3519,3699,180]
[3519,3695,176]
[3519,3714,195]
[3519,3736,217]
[3641,3699,58]
[3553,3641,88]
[3632,3641,9]
[3535,3641,106]
[3625,3641,16]
[3553,3587,34]
[3553,3790,237]
[3553,3653,100]
[3553,3660,107]
[3552,3553,1]
[3530,3660,130]
[3546,3660,114]
[3507,3660,153]
[3514,3530,16]
[3530,3674,144]
[3530,3729,199]
[3530,3552,22]
[3514,3708,194]
[3514,3515,1]
[3515,3615,100]
[3558,3615,57]
[3615,3616,1]
[3615,3692,77]
[3608,3615,7]
